In [1]:
import pybaseball as pyb
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import matplotlib.colors as colors
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', None)

# Batter Line-Up Effect

### ***'조선의 4번 타자 이대호'***
   
4번 타자는 흔히 팀을 대표하는 강타자를 의미한다. 왜 하필 4번 타자일까? 
   
야구에서 공격하는 팀은 9명의 타자가 1-9번의 순번을 가지며 차례대로 타석에 들어선다. 보통 1-2번 타자는 '테이블세터'라 하여 출루율이 높은 선수들을 배치한다. 그리고 3-5번 타자는 중심 타선으로, 클린업 트리오라고 불리며 장타율이 높은 타자들을 배치한다. **이런 배치가 효율적인 것일까?** 각 타순이 타점을 기록하는 것에 영향을 미치는지에 대해 분석하여보았다. 
   
야구에서 타점(RBI, Run Batted In)은 타자가 타격을 통해 루 상에 나가있는 주자들을 불러들여 득점에 기여한 정도를 보여주는 지표이다. 타점 지표는 타자의 타격 능력도 중요하지만, 타자가 **타석에 들어섰을 때 루 상에 주자가 얼마나 나가있는지 또한 큰 영향을 미친다**. 

19~21시즌의 메이저리그 데이터를 이용하여 **각 타순별로 맞이한 주자 상황에 차이가 있는지 확인**해보았다. 그리고 **한명의 가상의 타자가 각 타순에 배치되었을 때, 획득할 수 있는 타점에 대해 계산하여 비교**해보았다.
   
### **1. [Data](#data)**
- 19~21시즌 Statcast데이터를 불러왔다.
- 타석을 순서대로 정렬하여 타순을 부여하고, 각 타석에서의 주자 상황을 표시하였다.
- 타수에서 제외되는 데이터들(사사구, 희생타 등)을 제외시켰다.

### **2. [Average Number of BaseRunner by LineUp](#baserunner)**
- 각 타순의 타자가 타석에 들어섰을 때, 각 루에 주자가 있을 확률을 계산하였다.
       
### **3. [RBI Probability by ( Hit Type & BaseRunner )](#rbi_prob)**
- 각 루에 있는 주자들이 타격 결과에 의해 득점할 확률을 계산하여 표로 표현하였다.
    - ex1) 1루에 주자가 있을 경우, 2루타를 쳤을 때 1루 주자가 득점하여 타점으로 기록될 확률 
    - ex2) 2루에 주자가 있을 경우, 1루타를 쳤을 때 2루 주자가 득점하여 타점으로 기록될 확률

### **4. [Expected RBI Difference by Batting Order ( hypothetical player )](#rbi_diff)**
- 타순의 영향을 확인하기 위해, 가상의 선수를 설정하였다.
- 가상의 선수가 모두 똑같은 타율과 장타율 성적을 기록한다고 가정하였을 때, 각 타순에서 몇점의 타점을 기록하였을지 계산해보았다.

----

<a id="data"></a>
# 19~21 MLB Data

In [2]:
mlb_data=pyb.statcast('2019-03-20', '2021-10-03')

This is a large query, it may take a moment to complete
Skipping offseason dates
Skipping offseason dates


100%|██████████| 525/525 [06:43<00:00,  1.30it/s]


In [3]:
event_data = mlb_data[mlb_data['events'].notnull()]

In [4]:
def calculation(event):
    total_game = pd.DataFrame()
    game_id = np.array(event['game_pk'].unique())
    for i in game_id:
        # 게임별 데이터 분리
        game = event_data[event_data['game_pk']==i]
       
        # 타석 순으로 정리
        game = game.sort_values(by=['at_bat_number'])
        
        # 1루, 2루, 3루에 주자가 있을 경우 1, 없을 경우 0으로
        game['Start1'] = np.where(game['on_1b'].isnull(),0,1)
        game['Start2'] = np.where(game['on_2b'].isnull(),0,1)
        game['Start3'] = np.where(game['on_3b'].isnull(),0,1)
        
        # 위의 주자 상황을 한번에 표시
        game['Start_State'] = (game['Start1'].astype(str) + game['Start2'].astype(str) + game['Start3'].astype(str))
        
        # 이닝 초, 말로 구분하여 각 팀을 구분
        top = game[game['inning_topbot']=='Top']
        bot = game[game['inning_topbot']=='Bot']
        
        # 타석 순서대로 순번을 부여
        top['order'] = range(1, len(top)+1 ,1)
        top['batting_order'] = top['order']%9
        top['batting_order'] = np.where(top['batting_order']==0, 9, top['batting_order'])
        bot['order'] = range(1, len(bot)+1 ,1)
        bot['batting_order'] = bot['order']%9
        bot['batting_order'] = np.where(bot['batting_order']==0, 9, bot['batting_order'])

        top_bot = pd.concat([top, bot])

        total_game = pd.concat([total_game, top_bot])
        
    return total_game

In [5]:
df = calculation(event_data)

In [6]:
# 타수에서 제외되는 데이터 제거
not_ab = ['walk', 'sac_bunt', 'sac_fly', 'hit_by_pitch']
ab_data = df[~df['events'].isin(not_ab)]

----

<a id="baserunner"></a>
# Average Number of BaseRunner by LineUp

In [7]:
# 총 주자 숫자 
ab_data['baserunners_on'] = ab_data['Start1'] + ab_data['Start2'] + ab_data['Start3']

# 타순 별로 루에 주자가 몇명 있었는지 count
baserunner = ab_data.groupby(['batting_order']).agg({'Start_State': 'count', 'Start1': 'sum', 'Start2': 'sum', 'Start3': 'sum', 'baserunners_on': 'sum'}).reset_index()

# 위의 카운트를 총 타수로 나누어서 타순별 각 루에 주자가 있을 확률을 계산
baserunner['Start1'] = baserunner['Start1']/baserunner['Start_State']
baserunner['Start2'] = baserunner['Start2']/baserunner['Start_State']
baserunner['Start3'] = baserunner['Start3']/baserunner['Start_State']

# 모든 루에 대한 주자 숫자
baserunner['baserunners_on'] = baserunner['baserunners_on']/baserunner['Start_State']
baserunner.rename(columns={'Start_State': 'ab', 'Start1': 'runner_on_1b', 'Start2': 'runner_on_2b', 'Start3': 'runner_on_3b'}, inplace=True)
baserunner


,batting_order,ab,runner_on_1b,runner_on_2b,runner_on_3b,baserunners_on
0,1,50973,0.223432,0.151257,0.074294,0.448983
1,2,49548,0.292464,0.168725,0.077158,0.538347
2,3,48051,0.326445,0.191526,0.089093,0.607063
3,4,46907,0.339544,0.209052,0.099985,0.648581
4,5,46412,0.323322,0.204516,0.101547,0.629385
5,6,45393,0.316899,0.192056,0.096535,0.605490
6,7,44186,0.318698,0.191169,0.095143,0.605011
7,8,42817,0.312166,0.194642,0.093795,0.600603
8,9,40966,0.289362,0.191085,0.095469,0.575917


표를 살펴보면, **타순별 각 루에 주자에 있을 확률**을 확인할 수 있다. 1번 타자의 경우, 타석에 들어섰을 때, 1루에 주자가 있을 확률이 약 0.225, 2루에 주자가 있을 확률이 0.156, 3루에 주자가 있을 확률이 0.081이다. **각 루에 주자가 있을 확률들은 4-5번 타순으로 갈수록 증가하는 것을 확인**할 수 있었고, **6번부터 9번 타순은 조금 감소하여 비슷한 확률**들을 가지고 있는 것을 확인할 수 있었다.
   
또한 눈에 띄는 것은 타수였다. **총 타수는 1,2번 타자가 가장 많았고, 9번 타순으로 갈수록 감소**하는 모습을 보여주었다.

----

<a id="rbi_prob"></a>
# RBI Probability by ( Hit Type & BaseRunner )

In [8]:
ab_data['score_change'] = ab_data['post_bat_score'] - ab_data['bat_score']
hit = ['single', 'double', 'triple', 'home_run']
hit_df = ab_data[ab_data['events'].isin(hit)]

event_change = hit_df.groupby(['Start_State', 'Start1', 'Start2', 'Start3', 'events', 'score_change']).agg({'order': 'count'}).reset_index().rename(columns={'order': 'count'})

rbi_percentage = pd.DataFrame(['runner_on_1b', 'runner_on_2b', 'runner_on_3b'], columns=['state'])
for i in hit:
    # 'single', 'double', 'triple', 'home_run'에 대해 반복
    hit_type = event_change[event_change['events']==i]
    
    # 1루에 주자가 있을 경우
    total_1b = hit_type[hit_type['Start1']==1]
    # 타격에 의해 1루 주자가 홈에 들어온 경우
    run_1b = total_1b[((total_1b['Start1']==1) & (total_1b['Start2']==0) & (total_1b['Start3']==0) & (total_1b['score_change']>=1)) |
                     ((total_1b['Start1']==1) & (total_1b['Start2']==1) & (total_1b['Start3']==0) & (total_1b['score_change']>=2)) |
                     ((total_1b['Start1']==1) & (total_1b['Start2']==0) & (total_1b['Start3']==1) & (total_1b['score_change']>=2)) |
                     ((total_1b['Start1']==1) & (total_1b['Start2']==1) & (total_1b['Start3']==1) & (total_1b['score_change']>=3))]
    
    # 2루에 주자가 있을 경우
    total_2b = hit_type[hit_type['Start2']==1]
    # 타격에 의해 2루 주자가 홈에 들어온 경우
    run_2b = total_2b[((total_2b['Start1']==0) & (total_2b['Start2']==1) & (total_2b['Start3']==0) & (total_2b['score_change']>=1)) |
                     ((total_2b['Start1']==1) & (total_2b['Start2']==1) & (total_2b['Start3']==0) & (total_2b['score_change']>=1)) |
                     ((total_2b['Start1']==0) & (total_2b['Start2']==1) & (total_2b['Start3']==1) & (total_2b['score_change']>=1)) |
                     ((total_2b['Start1']==1) & (total_2b['Start2']==1) & (total_2b['Start3']==1) & (total_2b['score_change']>=2))]
    
    # 3루에 주자가 있을 경우
    total_3b = hit_type[hit_type['Start3']==1]
    # 타격에 의해 3루 주자가 홈에 들어온 경우
    run_3b = total_3b[total_3b['score_change']>=1]
    
    d = {'state': ['runner_on_1b', 'runner_on_2b', 'runner_on_3b'], i: [(run_1b['count'].sum()/total_1b['count'].sum()),
                                                                       (run_2b['count'].sum()/total_2b['count'].sum()),
                                                                       (run_3b['count'].sum()/total_3b['count'].sum())]}
    rbi_p = pd.DataFrame(data=d)
    rbi_percentage = pd.merge(rbi_percentage, rbi_p, on='state')
rbi_percentage.set_index('state', inplace=True)
rbi_percentage = rbi_percentage.T

In [9]:
rbi_percentage

state,runner_on_1b,runner_on_2b,runner_on_3b
single,0.009513,0.649229,0.993409
double,0.398868,0.988497,1.000000
triple,1.000000,1.000000,1.000000
home_run,1.000000,1.000000,1.000000


각 루에 있는 주자들이 타격 결과에 의해 득점할 확률을 계산하여 표로 표현하였다.
    
1루에 주자가 있을 경우, 1루타에 의해 1루 주자가 타점으로 연결될 확률은 약 0.9%였다. 2루타의 경우, 1루 주자가 타점으로 연결될 확률은 약 39.5%였다.   
2루에 주자가 있을 경우, 1루타에 의해 2루 주자가 타점으로 연결될 확률은 약 64.5%였다. 2루타의 경우, 2루 주자가 타점으로 연결될 확률은 약 98.9%였다.   
3루에 주자가 있을 경우, 1루타에 의해 3루 주자가 타점으로 연결될 확률은 약 99.3%였다. 2루타의 경우, 3루 주자가 타점으로 연결될 확률은 100%였다.   
   
3루타와 홈런의 경우, 어떤 루에 주자가 있던 주자가 득점할 확률은 100%였다.

----

<a id="rbi_diff"></a>
# Expected RBI Difference by Batting Order

위에서 각 타순별 맞이할 주자 상황 확률과 각 주자 상황에서의 타점 확률을 계산하였다. 계산된 것들을 이용하여 타순별 기대 타점 차이를 확인해보았다.

우선 각 타순에서 완벽하게 똑같은 성적을 내는 가상의 선수를 설정하였다. 이 가상의 타자는 타석에서 18% 확률로 Single를, 5% 확률로 Double를, 0.5% 확률로 Triple를, 그리고 5% 확률로 HomeRun을 기록하는 타자이다. 이 타자가 모든 타순에서 600 타수를 수행하였을 때 기대할 수 있는 타점을 계산해 비교해보았다.   
   
계산 방법은 다음과 같다. 이 가상의 타자가 1번 타순에 배치되었을 때, 기대할 수 있는 타점은
1. 가상의 타자는 600타수에서 single 108개, double 30개, triple 3개, home_run 30개를 기록한다.
2. 1번 타자일 경우, 1루에 주자가 있을 확률이 약 0.22, 2루에 주자가 있을 확률이 약 0.15, 3루에 주자가 있을 확률이 약 0.07이다.
    1. single로 1루 주자가 득점할 확률은 0.009, 2루 주자가 득점할 확률은 0.65, 3루 주자가 득점할 확률은 0.99이다.
        1. 타자가 친 single 개수(108) x 1루 주자가 있을 확률(0.22) x single로 1루 주자가 득점할 확률(0.009) = 0.21
        2. 타자가 친 single 개수(108) x 2루 주자가 있을 확률(0.15) x single로 2루 주자가 득점할 확률(0.65) = 10.53
        3. 타자가 친 single 개수(108) x 3루 주자가 있을 확률(0.07) x single로 3루 주자가 득점할 확률(0.99) = 7.45
    2. double로 1루 주자가 득점할 확률은 0.39, 2루 주자가 득점할 확률은 0.99, 3루 주자가 득점할 확률은 1이다.
        1. 타자가 친 double 개수(30) x 1루 주자가 있을 확률(0.22) x double로 1루 주자가 득점할 확률(0.39) = 2.57
        2. 타자가 친 double 개수(30) x 2루 주자가 있을 확률(0.15) x double로 2루 주자가 득점할 확률(0.99) = 4.46
        3. 타자가 친 double 개수(30) x 3루 주자가 있을 확률(0.07) x double로 3루 주자가 득점할 확률(1) = 2.1
    3. triple로 1루 주자가 득점할 확률은 1, 2루 주자가 득점할 확률은 1, 3루 주자가 득점할 확률은 1이다.
        1. 타자가 친 triple 개수(3) x 1루 주자가 있을 확률(0.22) x triple로 1루 주자가 득점할 확률(1) = 0.66
        2. 타자가 친 triple 개수(3) x 2루 주자가 있을 확률(0.15) x triple로 2루 주자가 득점할 확률(1) = 0.45
        3. 타자가 친 triple 개수(3) x 3루 주자가 있을 확률(0.07) x triple로 3루 주자가 득점할 확률(1) = 0.21
    4. home_run으로 1루 주자가 득점할 확률은 1, 2루 주자가 득점할 확률은 1, 3루 주자가 득점할 확률은 1이다.
        1. 타자가 친 home_run 개수(30) x 1루 주자가 있을 확률(0.22) x triple로 1루 주자가 득점할 확률(1) = 6.6
        2. 타자가 친 home_run 개수(30) x 2루 주자가 있을 확률(0.15) x triple로 2루 주자가 득점할 확률(1) = 4.5
        3. 타자가 친 home_run 개수(30) x 3루 주자가 있을 확률(0.07) x triple로 3루 주자가 득점할 확률(1) = 2.1
    5. home_run의 경우 타자 자신도 타점이 된다.
        1. 타자가 친 home_run 개수(30) = 30 
3. 모든 값들을 더하면 가상의 타자의 1번 타순에서의 기대 타점이 계산된다.(약 72.84)
   
1번 타순에서의 기대 타점은 72.84로 계산되고, 이 계산을 모든 타순에 대하여 수행하였다.

In [10]:
single_prob=0.18
double_prob=0.05
triple_prob=0.005
home_run_prob=0.05

ab= 600

single_count = single_prob*ab
double_count = double_prob*ab
triple_count = triple_prob*ab
home_run_count = home_run_prob*ab

In [11]:
rbi_prob = baserunner[['batting_order','runner_on_1b','runner_on_2b','runner_on_3b']].set_index('batting_order')

# Single에 의해 각 루의 주자가 득점할 평균 점수
rbi_prob[['single_rbi_1b', 'single_rbi_2b', 'single_rbi_3b']] = rbi_prob[['runner_on_1b','runner_on_2b','runner_on_3b']] * rbi_percentage.iloc[0] * single_count
# Double에 의해 각 루의 주자가 득점할 평균 점수
rbi_prob[['double_rbi_1b', 'double_rbi_2b', 'double_rbi_3b']] = rbi_prob[['runner_on_1b','runner_on_2b','runner_on_3b']] * rbi_percentage.iloc[1] * double_count
# Triple에 의해 각 루의 주자가 득점할 평균 점수
rbi_prob[['triple_rbi_1b', 'triple_rbi_2b', 'triple_rbi_3b']] = rbi_prob[['runner_on_1b','runner_on_2b','runner_on_3b']] * rbi_percentage.iloc[2] * triple_count
# HomeRun에 의해 각 루의 주자가 득점할 평균 점수
rbi_prob[['hr_rbi_1b', 'hr_rbi_2b', 'hr_rbi_3b']] = rbi_prob[['runner_on_1b','runner_on_2b','runner_on_3b']] * rbi_percentage.iloc[3] * home_run_count

# 1루 주자를 타점으로 연결한 모든 경우
rbi_prob['1b_rbi_600'] = rbi_prob['single_rbi_1b'] + rbi_prob['double_rbi_1b'] + rbi_prob['triple_rbi_1b'] + rbi_prob['hr_rbi_1b']
# 2루 주자를 타점으로 연결한 모든 경우
rbi_prob['2b_rbi_600'] = rbi_prob['single_rbi_2b'] + rbi_prob['double_rbi_2b'] + rbi_prob['triple_rbi_2b'] + rbi_prob['hr_rbi_2b']
# 3루 주자를 타점으로 연결한 모든 경우 
rbi_prob['3b_rbi_600'] = rbi_prob['single_rbi_3b'] + rbi_prob['double_rbi_3b'] + rbi_prob['triple_rbi_3b'] + rbi_prob['hr_rbi_3b']

exp_rbi = rbi_prob[['1b_rbi_600', '2b_rbi_600',  '3b_rbi_600']]

# 홈런의 경우 batter 자체가 타점으로 기록
exp_rbi['batter_600'] = home_run_count

# 1루 주자, 2루 주자, 3루 주자, 타자 본인을 모두 종합한 타점
exp_rbi['total_RBI_600'] = exp_rbi['1b_rbi_600'] + exp_rbi['2b_rbi_600'] + exp_rbi['3b_rbi_600'] + exp_rbi['batter_600']
exp_rbi = exp_rbi.reset_index()

exp_rbi_color = (exp_rbi.style
            .set_caption('Expected RBI by Batting Order (hypothetical player) 600 At Bats')
            .background_gradient(cmap='coolwarm', subset=exp_rbi.columns[5]))

exp_rbi_color

,batting_order,1b_rbi_600,2b_rbi_600,3b_rbi_600,batter_600,total_RBI_600
0,1,10.276400,20.082577,12.651431,30.000000,73.010408
1,2,13.451410,22.401928,13.139012,30.000000,78.992351
2,3,15.014310,25.429174,15.171459,30.000000,85.614942
3,4,15.616795,27.756167,17.026278,30.000000,90.399240
4,5,14.870660,27.153930,17.292258,30.000000,89.316848
5,6,14.575269,25.499592,16.438721,30.000000,86.513582
6,7,14.658018,25.381838,16.201774,30.000000,86.241629
7,8,14.357566,25.842975,15.972100,30.000000,86.172641
8,9,13.308740,25.370704,16.257314,30.000000,84.936758


계산 결과를 데이터프레임으로 표현하였다. 가장 오른쪽이 각 타순에서의 기대 타점값이다. 가상의 선수가 각 타순에서 600의 타수를 소화하였을 때, **1번 타순에서는 72.84의 타점을 기대할 수 있는 반면에 4번 타순에서는 90.19의 타점을 기대**할 수 있었다.
   
전체적으로 보면 1,2번 타순에서는 타점 기대값이 70대로 가장 낮았고, 4-5번 타순까지 타점 기대값이 90으로 커지다가 6-9번 타순은 약 85점으로 비슷한 모습을 보여주었다.
    
위의 계산된 타점 기대값은 각 타순에서 **똑같은 타수(600타수)를 소화하였을 때의 타점 기대값**이다. 그러나 처음 계산된 표에 의하면 각 타순별 총 타수는 차이를 나타내었다. **1,2번 타순이 가장 많이 타수를 소화하였고, 하위 타순으로 갈수록 소화하는 타수가 줄어들었다.**
   
이 타순별 타수의 차이를 보정하기 위해, 4번 타순의 타수를 기준(600타수)으로 나머지 타순의 타수를 조정하였다.

In [12]:
# 타수 보정
baserunner['ab_adj'] = (baserunner['ab']/baserunner['ab'].iloc[3])*600
adjust_ab = baserunner[['batting_order', 'ab', 'ab_adj']]
adjust_ab

,batting_order,ab,ab_adj
0,1,50973,652.009295
1,2,49548,633.781738
2,3,48051,614.633210
3,4,46907,600.000000
4,5,46412,593.668322
5,6,45393,580.634021
6,7,44186,565.194960
7,8,42817,547.683715
8,9,40966,524.007078


4번 타순의 타수를 600으로 고정하고, 다른 타순의 타수를 보정하였다. 계산 결과 1번 타순은 652타수, 2번 타순은 634타수로 가장 높았고, 이후로 타순이 뒤로 갈수록 타수가 줄어들어 9번 타순의 경우 타수가 약 524타수가 계산되었다.
   
보정 계산된 각 타순의 타수를 적용하여 타순별 타점 기대값을 계산하였다.

In [13]:
exp_rbi_adj_ = pd.merge(adjust_ab, exp_rbi, on='batting_order')
exp_rbi_adj_['adj_ratio'] = exp_rbi_adj_['ab_adj']/exp_rbi_adj_['ab_adj'].iloc[3]
exp_rbi_adj_[['1b_rbi_adj', '2b_rbi_adj', '3b_rbi_adj', 'batter_adj', 'total_RBI_adj']] = exp_rbi_adj_[['1b_rbi_600', '2b_rbi_600', '3b_rbi_600', 'batter_600', 'total_RBI_600']].multiply(exp_rbi_adj_['adj_ratio'], axis='index')

exp_rbi_adj = exp_rbi_adj_[['batting_order', 'ab_adj', '1b_rbi_adj', '2b_rbi_adj', '3b_rbi_adj', 'batter_adj', 'total_RBI_adj']]
exp_rbi_adj

exp_rbi_adj = (exp_rbi_adj.style
            .set_caption('Expected RBI by Batting Order (hypothetical player) adjusted AB')
            .background_gradient(cmap='coolwarm', subset=exp_rbi_adj.columns[6]))

exp_rbi_adj

,batting_order,ab_adj,1b_rbi_adj,2b_rbi_adj,3b_rbi_adj,batter_adj,total_RBI_adj
0,1,652.009295,11.167180,21.823378,13.748085,32.600465,79.339107
1,2,633.781738,14.208763,23.663222,13.878777,31.689087,83.439849
2,3,614.633210,15.380489,26.049358,15.541471,30.731661,87.702978
3,4,600.000000,15.616795,27.756167,17.026278,30.000000,90.399240
4,5,593.668322,14.713733,26.867380,17.109776,29.683416,88.374306
5,6,580.634021,14.104828,24.676551,15.908135,29.031701,83.721215
6,7,565.194960,13.807729,23.909478,15.261935,28.259748,81.238890
7,8,547.683715,13.105675,23.589628,14.579432,27.384186,78.658920
8,9,524.007078,11.623123,22.157380,14.198246,26.200354,74.179104


타수를 보정하여 계산한 결과, 4번 타순의 타점 기대값은 그대로 약 90.2점이었다. 반면에 70점대 초반을 기록했던 1번 타순의 타점 기대값은 타수의 보정을 받아 약 79.1점으로 올랐고, 6-9번 타순의 타점 기대값은 많이 떨어졌다.
    
최종적으로 보정된 타수가 적용된 각 타순의 타점 기대값이 계산되었다. 계산된 값들을 살펴보면, **모든 타순의 타자들이 같은 타격 능력을 가지고 있더라도 4번 타순에 배치되었을 경우 타점을 가장 많이 생성**할 수 있다고 해석할 수 있다. 그만큼 **4번의 타순이 득점을 만들 수 있는 기회가 가장 많다**는 것이고, 이것이 4번 타순에 팀을 대표하는 강타자를 배치시키는 이유라고 볼 수 있다.
    
중심 타선이라고 불리는 3-5번 타순이 역시나 가장 높은 타점 기대값을 가지고 있었다. 가장 낮은 9번 타순의 경우 4번 타순과 무려 약 17점의 타점 기대값 차이가 있었다. 이런 차이를 통해, 타자들의 타점을 평가할 때 그 **타자의 타순도 고려**하여야 정확한 평가가 이루어 진다는 것을 확인할 수 있었다.

### Reference

*팬그래프 (www.fangraphs.com)*